In [74]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from pycaret.regression import setup, compare_models
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import KFold, cross_val_score

In [94]:
suffix = "12"
df = pd.read_csv(f'data/preprocessed{suffix}.csv')
df.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SalePrice
0,60,65.0,9.042040,7.0,5.0,2003.0,2003.0,1.837880,6.561031,0.0,...,0,0,1,0,0,0,0,1,0,208500.0
1,20,80.0,9.169623,6.0,8.0,1976.0,1976.0,0.000000,6.886532,0.0,...,0,0,1,0,0,0,0,1,0,181500.0
2,60,68.0,9.328212,7.0,5.0,2001.0,2002.0,1.807264,6.188264,0.0,...,0,0,1,0,0,0,0,1,0,223500.0
3,70,60.0,9.164401,7.0,5.0,1915.0,1970.0,0.000000,5.379897,0.0,...,0,0,1,1,0,0,0,0,0,140000.0
4,60,84.0,9.565284,8.0,5.0,2000.0,2000.0,1.925822,6.486161,0.0,...,0,0,1,0,0,0,0,1,0,250000.0


# store X and y

In [76]:
y = df.iloc[:, -1].values
X = df.iloc[:, :-1].values
X.shape, y.shape

((2919, 823), (2919,))

# scale X

In [77]:
X = StandardScaler().fit_transform(X)

# split test and train

In [78]:
train_idx = 1460
X_train, X_test = X[:train_idx], X[train_idx:]
y_train, _ = y[:train_idx], y[train_idx:]

# take log of y train

In [79]:
y_train = np.log(y_train)

In [80]:
df_train = pd.DataFrame(np.concatenate((X_train, y_train.reshape(-1, 1)), axis=1))
df_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823
0,60.0,65.0,9.042040,7.0,5.0,2003.0,2003.0,1.837880,6.561031,0.0,150.0,856.0,2.048136,6.751101,7.444833,0.526589,0.000000,1.098612,0.693147,3.0,0.423036,2.197225,0.000000,2003.0,2.0,548.0,0.000000,4.127134,0.000000,0.0,0.0,1.047198,2008.0,4225.0,587.732604,455.0,325.0,130195.0,130195.0,119.462200,426.466993,0.0,9750.0,55640.0,133.128863,438.821595,483.914163,34.228287,0.000000,71.409799,45.054567,195.0,27.497331,142.819598,0.000000,130195.0,130.0,35620.0,0.000000,268.263735,0.000000,0.0,0.0,68.067843,130520.0,81.758488,63.294280,45.210200,18111.206233,18111.206233,16.618185,59.325102,0.0,1356.306008,7739.986288,18.519331,61.043730,67.316481,4.761439,0.000000,9.933696,6.267465,27.126120,3.825107,19.867393,0.000000,18111.206233,18.084080,4955.037951,0.000000,37.317714,0.000000,0.0,0.0,9.468802,18156.416434,49.0,35.0,14021.0,14021.0,12.865160,45.927215,0.0,1050.0,5992.0,14.336954,47.257710,52.113833,3.686123,0.000000,7.690286,4.852030,21.0,2.961251,15.380572,0.000000,14021.0,14.0,3836.0,0.000000,28.889941,0.000000,0.0,0.0,7.330383,14056.0,25.0,10015.0,10015.0,9.189400,32.805153,0.0,750.0,4280.0,10.240682,33.755507,37.224166,2.632945,0.000000,5.493061,3.465736,15.0,2.115179,10.986123,0.000000,10015.0,10.0,2740.0,0.000000,20.635672,0.000000,0.0,0.0,5.235988,10040.0,4012009.0,4012009.0,3681.273637,13141.744424,0.0,300450.0,1714568.0,4102.417104,13522.456242,14912.001048,1054.757835,0.000000,2200.520414,1388.373803,6009.0,847.340822,4401.040828,0.000000,4012009.0,4006.0,1097644.0,0.000000,8266.650173,0.000000,0.0,0.0,2097.536753,4022024.0,4012009.0,3681.273637,13141.744424,0.0,300450.0,1714568.0,4102.417104,13522.456242,14912.001048,1054.757835,0.000000,2200.520414,1388.373803,6009.0,847.340822,4401.040828,0.000000,4012009.0,4006.0,1097644.0,0.000000,8266.650173,0.000000,0.0,0.0,2097.536753,4022024.0,3.377803,12.058387,0.0,275.682000,1573.225279,3.764229,12.407714,13.682710,0.967807,0.0,2.019118,1.273921,5.513640,0.777489,4.038235,0.000000,3681.273637,3.675760,1007.158239,0.000000,7.585178,0.0,0.0,0.0,1.924623,3690.463037,43.047123,0.0,984.154600,5616.242250,13.437885,44.2941

# rename target col

In [81]:
# rename last column to target
df_train.rename(columns={df_train.columns[-1]: 'target'}, inplace=True)
df_train["target"]

0       12.247694
1       12.109011
2       12.317167
3       11.849398
4       12.429216
          ...    
1455    12.072541
1456    12.254863
1457    12.493130
1458    11.864462
1459    11.901583
Name: target, Length: 1460, dtype: float64

# setup pycaret

In [82]:
s = setup(data=df_train, target='target')

,Description,Value
0,Session id,6965
1,Target,target
2,Target type,Regression
3,Data shape,"(1460, 824)"
4,Train data shape,"(1021, 824)"
5,Test data shape,"(439, 824)"
6,Numeric features,823
7,Preprocess,True
8,Imputation type,simple
9,Numeric imputation,mean


# Compare different models

In [85]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,0.0915,0.0180,0.1323,0.8887,0.0103,0.0077,2.0930
lightgbm,Light Gradient Boosting Machine,0.0913,0.0181,0.1326,0.8880,0.0103,0.0076,0.9170
et,Extra Trees Regressor,0.0945,0.0196,0.1385,0.8796,0.0108,0.0079,1.7670
rf,Random Forest Regressor,0.0949,0.0196,0.1387,0.8794,0.0108,0.0080,2.3560
omp,Orthogonal Matching Pursuit,0.0855,0.0197,0.1342,0.8791,0.0103,0.0072,0.0500
xgboost,Extreme Gradient Boosting,0.1048,0.0231,0.1501,0.8571,0.0117,0.0088,1.8910
lasso,Lasso Regression,0.0905,0.0263,0.1467,0.8412,0.0112,0.0076,0.1150
ada,AdaBoost Regressor,0.1178,0.0264,0.1615,0.8379,0.0125,0.0098,0.8770
en,Elastic Net,0.0916,0.0308,0.1530,0.8155,0.0115,0.0077,0.1200
lr,Linear Regression,0.1217,0.0298,0.1711,0.8117,0.0133,0.0102,0.0810


In [88]:
result = cross_val_score(best, 
                        X_train, 
                        y_train, 
                        scoring="neg_mean_squared_error",
                        cv=KFold(n_splits=10))
print(np.exp(np.sqrt(-result)).mean())

1.1400376180672283


In [89]:
pred = np.exp(best.predict(X_test))
pd.DataFrame(pred, 
            index=range(1461, len(df)+1), 
            columns=['SalePrice']).reset_index().rename(columns={'index': 'id'}).\
                to_csv(f'data/submission{suffix}_best.csv', index=False)

# train ensemble of best models 

In [67]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import BayesianRidge, OrthogonalMatchingPursuit, LinearRegression, Lasso, Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Bagging ensemble

In [68]:
models = {
    "gbr": GradientBoostingRegressor(verbose=0),
    "rf": RandomForestRegressor(),
    "xgb": XGBRegressor(),
    "lgbm": LGBMRegressor(),
    "et": ExtraTreesRegressor(),
    "br": BayesianRidge(),
    "omp": OrthogonalMatchingPursuit(),
}

# Evaluate

In [69]:
results = {}
for name, model in models.items():
    result = cross_val_score(model, 
                            X_train, 
                            y_train, 
                            scoring="neg_mean_squared_error",
                            cv=KFold(n_splits=10))
    results[name] = np.exp(np.sqrt(-result))

In [70]:
results

{'gbr': array([1.13605481, 1.11290607, 1.11881022, 1.19309385, 1.16995792,
        1.12717144, 1.14129604, 1.11635698, 1.13679067, 1.14575931]),
 'rf': array([1.1506536 , 1.11996163, 1.13269635, 1.18766705, 1.17685496,
        1.1305786 , 1.13390794, 1.12806345, 1.16677043, 1.15493304]),
 'xgb': array([1.16505496, 1.13470109, 1.16390057, 1.20583595, 1.1756001 ,
        1.14220334, 1.1616369 , 1.13218538, 1.14695166, 1.17433248]),
 'lgbm': array([1.13734728, 1.11305822, 1.12109834, 1.18623983, 1.17283698,
        1.12227567, 1.13192154, 1.11575235, 1.14459544, 1.15264308]),
 'et': array([1.14227902, 1.11448389, 1.13231433, 1.18086023, 1.1727946 ,
        1.11913407, 1.12905718, 1.11802082, 1.1476179 , 1.14763828]),
 'br': array([1.11939883, 1.10163111, 1.11191488, 1.18333471, 1.14887638,
        1.13620639, 1.12157118, 1.10356167, 1.24016792, 1.13784051]),
 'omp': array([1.12058136, 1.10528201, 1.11255767, 1.18669344, 1.15994902,
        1.16590421, 1.12968398, 1.10848162, 1.24121157, 1

In [71]:
means = []
for name, result in results.items():
    means.append(result.mean())
    print(f"{name}: {result.mean():.5f}")
top_model = [name for _,name in sorted(zip(means, models.keys()))]
print(top_model)

gbr: 1.13982
rf: 1.14821
xgb: 1.16024
lgbm: 1.13978
et: 1.14042
br: 1.14045
omp: 1.14706
['lgbm', 'gbr', 'et', 'br', 'omp', 'rf', 'xgb']


# Combine predictions

In [72]:
n_models = 2
pred = np.zeros(X_test.shape[0])
for model in top_model[:n_models]:
    models[model].fit(X_train, y_train)
    pred += np.exp(models[model].predict(X_test)) / n_models

# Save submission

In [73]:
y_submission = pred
pd.DataFrame(y_submission, 
            index=range(1461, len(df)+1), 
            columns=['SalePrice']).reset_index().rename(columns={'index': 'id'}).to_csv(f'data/submission{suffix}.csv', index=False)